In [56]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as pyplot
import xlrd
import seaborn as sb
import seaborn as sns; sns.set()

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn import linear_model
from sklearn import svm

import tensorflow as tf

In [ ]:
columns = ["cotacao","P/L","P/VP","PSR", "div.yield", 
           "P/ativo","P/cap.giro","P/EBIT","P/ativ_circ.liq", "EV/EBIT", 
           "EV/EBITDA","mrg_ebit","mrg.Liq","liq.corr","ROIC", 
           "ROE","liq.2meses","patrim.liq", "div.brut/patrim","cresc.rec.5a"]

data = pd.read_csv("https://raw.githubusercontent.com/GuilhermehChaves/stock-market/main/data.csv")
data.tail()

In [ ]:
data.hist(["cotacao","P/L","P/VP","PSR", "div.yield", 
           "P/ativo","P/cap.giro","P/EBIT","P/ativ_circ.liq", "EV/EBIT", 
           "EV/EBITDA","mrg_ebit","mrg.Liq","liq.corr","ROIC", 
           "ROE","liq.2meses","patrim.liq", "div.brut/patrim","cresc.rec.5a"], figsize = (15,15))

In [ ]:
data.query("papel == 'BIDI4'").values

# Correlação

In [129]:
correlation = (pd.DataFrame(data, columns = columns)).corr()

In [ ]:
import numpy

fig = plt.figure(figsize = (45,45))
ax = fig.add_subplot(111)
fig.subplots_adjust(top = 0.6)
cax = ax.matshow(correlation, vmin = -1, vmax = 1, cmap = "hot")
fig.colorbar(cax)
ticks = numpy.arange(0,20,1)

ax.set_xticks(ticks)
ax.set_yticks(ticks)

ax.set_xticklabels(columns, size = '12')
ax.set_yticklabels(columns, size = '12')
plt.show()

# Padronização

In [131]:
scaler_x = MinMaxScaler()
data[columns[1:]] = scaler_x.fit_transform(data[columns[1:]])
data.tail()

scaler_y = MinMaxScaler()
data[['cotacao']] = scaler_y.fit_transform(data[['cotacao']])

x = data.drop('cotacao', axis = 1)
y = data.cotacao

# Treinamento

In [132]:
columns_predictors = columns[1:]
cols = [tf.feature_column.numeric_column(key = c) for c in columns_predictors]
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.3)

In [133]:
training_function = tf.compat.v1.estimator.inputs.pandas_input_fn(x = x_train, y = y_train, batch_size = 32, num_epochs = None, shuffle = True)
test_function = tf.compat.v1.estimator.inputs.pandas_input_fn(x = x_test, y = y_test, batch_size = 32, num_epochs = 1000, shuffle = False)

In [ ]:
regressor = tf.estimator.LinearRegressor(feature_columns = cols)
regressor.train(input_fn = training_function, steps = 10000)

In [ ]:
training_metrics = regressor.evaluate(input_fn = training_function, steps = 10000)
test_metrics = regressor.evaluate(input_fn = test_function, steps = 10000)

# Previsões

In [136]:
forecast_function = tf.compat.v1.estimator.inputs.pandas_input_fn(x = x_test, shuffle = False)
forecasts = regressor.predict(input_fn = forecast_function)

In [ ]:
forecast_values = []

for p in regressor.predict(input_fn = forecast_function):
    forecast_values.append(p['predictions'])

forecast_values

## Desnormalizando dados

In [138]:
forecast_values = np.asarray(forecast_values).reshape(-1, 1)
forecast_values = scaler_y.inverse_transform(forecast_values)

In [139]:
y_test2 = y_test.values.reshape(-1, 1)
y_test2 = scaler_y.inverse_transform(y_test2)

In [140]:
from sklearn.metrics import mean_absolute_error

meanAbsoluteError = mean_absolute_error(y_test2, forecast_values)
print(meanAbsoluteError)

63.315436460343044


In [141]:
df = pd.DataFrame()
df['real'] = pd.Series(y_test2.flatten())
df['previsto'] = pd.Series(forecast_values.flatten())

In [ ]:
df.plot()